# Generate several sets of normal and non-normal samples <a class="tocSkip">

In this notebook, we create several datasets consisting of normal and non-normal samples. The simulated sets are named $\mathcal{A}-\mathcal{G}$. The real-world sets are named $\mathcal{R}_{height}$ and $\mathcal{R}_{earthquake}$. 

## Set up the environment

In [1]:
import os
import pathlib
import pickle

from pathlib import Path
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D
from sklearn.model_selection import train_test_split

import util
from ipynb.fs.defs.descriptor_based_neural_networks import traverse_and_save

Read the configuration.

In [2]:
# Define the path to the configuration dictionary
config_path = 'configuration.p'

# Load the configuration dictionary if it exists
with open(config_path, 'rb') as f:
    configuration = pickle.load(f)
    
# Get the path to the directory to which the datasets will be stored
data_directory_path = configuration['data']['directory_path']

Create a dictionary to store figures before saving them.

In [3]:
figures = {}

## Set $\mathcal{A}$

The samples consist of $10, 20, \ldots, 100$ elements. 

The normal samples are drawn from a a normal distributions $N(\mu,\sigma^2)$ whose location parameter ($\mu$) is randomly selected from the range $[-100,100]$ and the standard deviation is randomly drawn from the range $[1, 20]$. For each $n$, a total of $L$ normal distributions are defined and a sample of size $n$ is drawn from each of them.

The non-normal samples are drawn from the Pearson family of distributions. Each distribution is specified by its first four moments. The mean and standard deviation are determined the same as for the normal samples. They are combined with the skewness ($s$) and kurtosis ($k$) that range over $\{\}$ and $\{\}$ and fulfill the following two conditions: (1) $k - s^2 - 1 \geq 0$ and (2) $\neg(s=0 \land k=3)$. The first condition is a limitation known from theory. The second requirement is there to ensure that those non-normal distributions are sufficiently different from the normal ones, since for normal distributions it holds that $s = 0$ and $k = 3$. A sample is drawn from each such distribution.

The set is balanced. It contains $13050$ normal and $13050$ non-normal samples.

In [4]:
# Create a function that generates datasets
def generate_dataset(n_range, s_range, k_range, M, verbose=True):
    # Generate non-normal samples
    nonnormal_samples = util.generate_pearson_nonnormal_samples(s_range, k_range, n_range, M)

    # Calculate L, the number of normal samples of the same size
    L = len(nonnormal_samples) // len(n_range)
            
    # Generate L normal samples of size n for each n in n_range
    normal_samples = util.generate_normal_samples(n_range, L)

    # Print how many samples were generated
    if verbose:
        print("Normal samples: ", len(normal_samples))
        print("Non-normal samples: ", len(nonnormal_samples))

    # Label the sets
    normal_samples = util.label_samples(normal_samples, 1)
    nonnormal_samples = util.label_samples(nonnormal_samples, 0)

    # Unify them
    all_samples = normal_samples + nonnormal_samples
    
    return all_samples

Use the function from the above cell to create dataset $\mathcal{A}$.

In [ ]:
# Define the ranges for the sample sizes 
# and non-normal skewness and kurtosis in the set A
n_range = range(10, 101, 10)
s_range = [x/10.0 for x in range(-300, 301, 5)] # skewness range -300, 301, 5;-150, 151, 5;-805, 810, 5
k_range = [x/10.0 for x in range(0, 401, 5)]   # kurtosis range 0, 401, 5;0, 201, 5;0, 1610, 5

# Let M denote the number of non-normal samples drawn from the same distribution.
# Since the set is created as balanced, M will influence the number of normal samples
# in the set too. See the function generate_dataset for details.
M = 1

# Generate set A
set_A = generate_dataset(n_range, s_range, k_range, M)

Describe the dataset, update the configuration dictionary, and save the set.

In [5]:
# Create a function to describe the set, save it and update the configuration dictionary
def describe_and_save(all_samples, set_name, n_range):
    global data_directory_path
    global configuration
    
    # Describe the set
    metadata = {
        'name' : set_name,
        'n_range' : n_range,
        'number_of_normal' : len([sample for sample in all_samples if sample[-1] == 1]),
        'number_of_nonnormal' : len([sample for sample in all_samples if sample[-1] == 0]),
        'path' : os.path.join(data_directory_path, '{}.data'.format(set_name))
    }

    # Add it to the configuration dictionary
    configuration['data']['datasets'][set_name] = metadata

    # Save the set to a file
    path = metadata['path']
    util.save_to_file(all_samples, path)
    print("Saved to the file", path)

In [ ]:
describe_and_save(set_A, 'A', n_range)

Define the functions for visualizing selected sample statistics.

In [ ]:
# Define the function that can estimate certain sample statistics such as standard deviation, skewness, kurtosis
# and so on.
def calculate(sample, statistic):
    if statistic == 'sd':
        return stats.tstd(sample)
    elif statistic == 'skewness':
        return stats.skew(sample)
    elif statistic == 'kurtosis':
        return stats.kurtosis(sample)
    elif statistic == 'mad':
        return stats.median_abs_deviation(sample)
    elif statistic == 'rkurtosis':
        # robust kurtosis
        median = np.median(sample)
        robust_moment = stats.median_abs_deviation([(median - x)**4 for x in sample])
        mad = stats.median_abs_deviation(sample)
        return robust_moment / max(mad**4, 0.1**8)
    elif statistic == 'rskewness':
        # robust skewness
        median = np.median(sample)
        robust_moment = stats.median_abs_deviation([(median - x)**3 for x in sample])
        mad = stats.median_abs_deviation(sample)
        return robust_moment / max(mad**3, 0.1**8)
    elif statistic == 'm5':
        sd = stats.tstd(sample)
        m5 = stats.moment(sample, moment=5)
        return m5/max(sd**5, 0.1**8)
    elif statistic == 'm6':
        sd = stats.tstd(sample)
        m5 = stats.moment(sample, moment=6)
        return m5/max(sd**6, 0.1**8)
    elif statistic == 'm7':
        sd = stats.tstd(sample)
        m5 = stats.moment(sample, moment=7)
        return m5/max(sd**7, 0.1**8)

# Create a function that calculates selected statistics of each sample in a given set of samples
def inspect_dataset(samples, labels=None, statistics=['skewness', 'kurtosis']):
    desc = {label: {statistic : [] for statistic in statistics} for label in [0, 1]}

    for i in range(len(samples)):
        if labels is None:
            sample = samples[i][:-1]
            label = samples[i][-1]
        else:
            sample = samples[i]
            label = labels[i]
        
        for statistic in statistics:
            desc[label][statistic].append(calculate(sample, statistic))
    
    return desc

In [ ]:
# Inspect set A
desc_A = inspect_dataset(set_A)

In [ ]:
# Plot the skewness and kurtosis of normal and non-normal sampes in set A

# Define a function for plotting
def plot_skewness_and_kurtosis(data_desc):
    colors = ['crimson', 'navy']
    figs = {}
    for label in data_desc:
        #ax = Axes3D(fig)
        if len(data_desc[label]['skewness']) > 0:
            fig = plt.figure(figsize=(10, 7))
            plt.scatter(data_desc[label]['skewness'], 
                        data_desc[label]['kurtosis'], alpha=0.2, color=colors[label])
                        #desc[label]['standard_deviation'])
            #plt.xlim(-2, 2)
            #plt.ylim(0, 10)
            figs[label] = fig
    return figs

In [ ]:
figs = plot_skewness_and_kurtosis(desc_A)
figures['A'] = figs

## Set $\mathcal{B}$

This set differs from $\mathcal{A}$ only in the sizes of the samples. They contain contain $5, 15, \ldots, 95$ elements. Everything else is completely the same as in the set $\mathcal{A}$.

In [ ]:
# Define the ranges for the sample sizes 
# and non-normal skewness and kurtosis in the set A
n_range = range(5, 101, 10) 
s_range = [x/10.0 for x in range(-300, 301, 5)] # skewness range -150, 151, 5;-805, 810, 5
k_range = [x/10.0 for x in range(0, 401, 5)]   # kurtosis range 0, 201, 5;0, 1610, 5

# Let M denote the number of non-normal samples drawn from the same distribution
M = 1

set_B = generate_dataset(n_range, s_range, k_range, M)
describe_and_save(set_B, 'B', n_range)

In [ ]:
figs = plot_skewness_and_kurtosis(inspect_dataset(set_B))
figures['B'] = figs

## Set $\mathcal{C}$

This set contains non-normal samples whose sizes are $10, 20, \ldots, 100$.

The non-normal distributions from which the samples are drawn are hand-picked and are usually used to assess the empirical power of normality tests. They are clssified into four groups. $G_1, G_2, G_3$ and $G_4$. See the paper for more details. For each sample size $n \in \left\{10, 20, \ldots, 100\right\}$, $L=10000$ samples are drawn from each group.

Define the distributions.

In [ ]:
# Define the distributions from group G1
logistic = lambda n: stats.logistic.rvs(size = n)
laplace = lambda n: stats.laplace.rvs(size = n)
t1 = lambda n: stats.t.rvs(1, size = n)
t3 = lambda n: stats.t.rvs(3, size = n)

# Define the distributions from group G2
gumbel1 = lambda n: list(np.random.gumbel(loc=0, scale=1, size=n))
gumbel2 = lambda n: list(np.random.gumbel(loc=0, scale=2, size=n))
gumbel3 = lambda n: list(np.random.gumbel(loc=0, scale=0.5, size=n))

# Define the distributions from group G3
expon = lambda n: stats.expon.rvs(loc = 1, size = n)
gamma1 = lambda n: list(np.random.gamma(2, scale = 1, size = n))
gamma2 = lambda n: list(np.random.gamma(0.5, scale = 1, size = n))
lognormal1 = lambda n: list(np.random.lognormal(mean = 0, sigma = 1, size = n))
lognormal2 = lambda n: list(np.random.lognormal(mean = 0, sigma = 2, size = n))
lognormal3 = lambda n: list(np.random.lognormal(mean = 0, sigma = 0.5, size = n))
weibull1 = lambda n: stats.weibull_min.rvs(0.5, scale = 1, size = n)
weibull2 = lambda n: stats.weibull_min.rvs(2, scale = 1, size = n)

# Define the distributions from group G4
uniform = lambda n: list(np.random.uniform(low = 0, high = 1, size = n))
beta1 = lambda n: np.random.beta(2, 2, size = n)
beta2 = lambda n: np.random.beta(0.5, 0.5, size = n)
beta3 = lambda n: np.random.beta(4, 4, size = n)

# Let groups be a dictionary containing distributions from four groups G1-G4
groups = {
    1 : [logistic, laplace, t1, t3],
    2 : [gumbel1, gumbel2, gumbel3],
    3 : [expon, gamma1, gamma2, lognormal1, lognormal2, lognormal3, weibull1, weibull2],
    4 : [uniform, beta1, beta2, beta3, ]
}

Generate the set, save it and update the configuration dictionary.

In [ ]:
# Define the range of sample sizes
n_range = range(10, 101, 10)

# Define L, which denotes how many samples with n elements
# will be drawn from each group.
L = 10000

set_C = {}

for g in groups:
    # Select a group
    group = groups[g]
    
    # Prepare the storage to hold the samples from this group
    samples = []
    
    # Draw the samples
    for n in n_range:
        # Initialize the counter of the samples generated so far in this group
        so_far = 0
        
        # Iterate over the distributions in the group until L samples are generated
        d = 0
        while so_far <= L:
            # Get the distribution whose turn is to generate a sample
            dist = group[d]
            
            # Generate a sample
            sample = dist(n)
            
            # Store it
            samples.append(sample)
            
            # Increase the counters
            so_far = so_far + 1
            d = d + 1
            
            # Return to the first distribution in the group and start over
            if d == len(group):
                d = 0
    
    # Label the samples as non-normal
    samples = util.label_samples(samples, 0)
    
    # Describe the set
    set_name = 'C-G{}'.format(g)
    filename = '{}.data'.format(set_name)
    
    group_metadata = {
        'name' : set_name,
        'n_range' : n_range,
        'number_of_normal' : 0,
        'number_of_nonnormal' : len(samples),
        'path' : os.path.join(data_directory_path, filename)
    }

    # Add it to the configuration dictionary
    configuration['data']['datasets'][set_name] = group_metadata
                
    # Save the samples
    path = group_metadata['path']
    util.save_to_file(samples, path)
    print("Saved {} to the file {}".format(set_name, path))
    
    set_C[set_name] = samples

Plot the empirical skewness and kurtosis of the samples drawn from the four groups in set $\mathcal{C}$.

In [ ]:
figures['C'] = {}
statistics = ['skewness', 'kurtosis']
for group_name in set_C:
    desc = inspect_dataset(set_C[group_name], statistics=statistics)
    for label in desc:
        if label == 0:
            fig = plt.figure()
            plt.scatter(desc[label][statistics[0]],
                        desc[label][statistics[1]],
                        alpha=0.2,
                        color='crimson')
            figures['C'][group_name] = fig

## Set $\mathcal{D}$

Create it the same way as the set $\mathcal{A}$.

In [ ]:
# Define the ranges for the sample sizes 
# and non-normal skewness and kurtosis 
n_range = range(10, 101, 10)
s_range = [x/10.0 for x in range(-300, 301, 5)] # skewness range -150, 151, 5;-805, 810, 5
k_range = [x/10.0 for x in range(0, 401, 5)]   # kurtosis range 0, 201, 5;0, 1610, 5

# Let M denote the number of non-normal samples drawn from the same distribution.
# Since the set is created as balanced, M will influence the number of normal samples
# in the set. See the function generate_dataset for details.
M = 1

# Generate and register the set
set_D = generate_dataset(n_range, s_range, k_range, M)
describe_and_save(set_D, 'D', n_range)

## Set $\mathcal{E}$

Create a set that contains normal samples.

In [7]:
n_range = range(5, 101, 5)
L = 10000

samples = util.generate_normal_samples(n_range, L)

set_E = util.label_samples(samples, 1)

describe_and_save(set_E, 'E', n_range)

Saved to the file data/E.data


## Set $\mathcal{F}$

This set will contain contaminated normal samples. It will be used to test for robustness.

We will create it by contaminating a sample from the standard normal $N(0, 1)$ with a small number of elements drawn from $N(\mu, \sigma^2)$. Each sample in set $\mathcal{F}$ comes from a mixture of the following form:

$$(1-\varepsilon)\underbrace{N(0, 1)}_{\text{core}}+\varepsilon \underbrace{N(\mu, \sigma^2)}_{\text{contaminator}}$$

where we set $\varepsilon$ to $0.05$. Depending on $\mu$ and $\sigma^2$, we distinguish between four types of contamination:
- left ($\mu = -3, \sigma^2=1$),
- right ($\mu = 3, \sigma^2=1$),
- symmetric ($\mu=0, \sigma^2=6$),
- and central ($\mu=0, \sigma^2=0.01$)

In [ ]:
# Define the contaminators
left_contaminator = lambda m: np.random.normal(-3, 1, m).tolist()
right_contaminator = lambda m: np.random.normal(3, 1, m).tolist()
symmetric_contaminator = lambda m: np.random.normal(0, 6, m).tolist()
central_contaminator = lambda m: np.random.normal(0, 0.01, m).tolist() 

contaminators = {
    'left' : left_contaminator,
    'right': right_contaminator,
    'symmetric': symmetric_contaminator,
    'central': central_contaminator
}

# Define how many samples of each size will be created for each contamination type.
L = 10000

# Prepare a dictionary to store the dataset
contaminated_samples = {}

# Create the dataset
for code in contaminators:
    contaminator = contaminators[code]
    contaminated_samples[code] = []
    
    for n in range(10, 101, 10):
        # determine the actual number of contaminated and core elements
        n_contaminated = max(1, int(0.05 * n))
        n_core = n - n_contaminated
        
        # Create L contaminated samples 
        for l in range(L):
            # Draw the core part 
            core = np.random.normal(0, 1, n_core).tolist()
            
            # Draw the contaminated part
            contamination = contaminator(n_contaminated)
            
            # Unify them
            contaminated_sample = core + contamination
            
            # Store the sample
            contaminated_samples[code].append(contaminated_sample)
        
    # Label the samples as non-normal
    contaminated_samples[code] = util.label_samples(contaminated_samples[code], 0)
                

In [ ]:
# Save the samples contaminated in the same way in the same file
for x in contaminated_samples:
    set_name = 'F-{}'.format(x)
    path = os.path.join(data_directory_path, '{}.data'.format(set_name))
    util.save_to_file(contaminated_samples[x], path)
    
    group_metadata = {
        'name' : set_name,
        'n_range' : range(10, 10, 101),
        'number_of_normal' : 0,
        'number_of_nonnormal' : len(contaminated_samples[x]),
        'path' : path
    }

    configuration['data']['datasets'][set_name] = group_metadata

Plot the skewness and kurtosis of the contaminated samples.

In [ ]:
figures['F'] = {}
for x in contaminated_samples:
    desc = inspect_dataset(contaminated_samples[x], statistics=['skewness', 'kurtosis'])

    for label in desc:
        if label == 0:
            fig = plt.figure()
            plt.scatter(desc[label]['skewness'],
                        desc[label]['kurtosis'],
                        alpha=0.2,
                        color='crimson')
            figures['F'][x] = fig

## Set $\mathcal{G}$

Another set that is same as $\mathcal{A}$, except that the samples have sizes $5, 10, 15, \ldots, 100$.

In [6]:
# Define the ranges for the sample sizes 
# and non-normal skewness and kurtosis in the set A
n_range = range(5, 101, 5) 
s_range = [x/10.0 for x in range(-300, 301, 5)] # skewness range -150, 151, 5;-805, 810, 5
k_range = [x/10.0 for x in range(0, 401, 5)]   # kurtosis range 0, 201, 5;0, 1610, 5

# Let M denote the number of non-normal samples drawn from the same distribution
M = 1

set_G = generate_dataset(n_range, s_range, k_range, M)
describe_and_save(set_G, 'G', n_range)

Normal samples:  26100
Non-normal samples:  26100
Saved to the file data/G.data


## Real-World Datasets

### Set $\mathcal{R}_{height}$
Create the set that will contain the samples of heights from a larger set of heights. The original data were taken from https://github.com/rmcelreath/rethinking/blob/master/data/Howell1.csv .

In [ ]:
# Load the data
df = pd.read_csv('data/Howell1.csv', sep=';')

For both sexes and each age window from $[18, 27], [19, 28]$ to $[79, 88]$, extract the corresponding heights from the original data and form a new sample of heights.

In [ ]:
figs = {}
sex = ['female', 'male']

age_limits = [(x, x + 9) for x in range(18, 80)]

samples = {}

for male in [0, 1]:
    for (min_age, max_age) in age_limits:
        mask = (df['male'] == male) & (df['age'] >= min_age) & (df['age'] < max_age)
        
        heights = list(df[mask]['height'].values)
        samples[(male, min_age, max_age)] = heights
        n = len(heights)
        
        description = df[mask]['height'].describe()
        m = description['mean']
        s = description['std']
        
        #fig = plt.figure(figsize=(10, 7))
        
        #divergences = []

        for i in range(100):
            random_sample = np.random.normal(m, s, n)
            #density = stats.gaussian_kde(random_sample)
            #divergence = stats.entropy(heights, random_sample)
            #divergences.append(divergence)
    
            random_sample.sort()
            #plt.hist(random_sample, bins=10, color='lavender', density=True, cumulative=True,
            #          histtype='step', linewidth=3)

        #plt.hist(heights, bins=10, color='navy', density=True, cumulative=True, histtype='step', linewidth=3)
 
        #legend_objects = [Line2D([0], [0], color='navy', lw=3), Line2D([0], [0], color='lavender', lw=3)]
        #plt.legend(legend_objects, ['Original EDF', 'Simulated EDFs'], fontsize=13, loc='upper left')
        #plt.xlabel('Height (cm)')
        #figs[(male, min_age, max_age)] = fig

In [ ]:
# Label samples as normal and save them
samples = util.label_samples(list(samples.values()), 1)

describe_and_save(samples, 'R_height', None)

In [ ]:
figs = {'{}_{}_{}_fig'.format(*key) : figs[key] for key in figs}

traverse_and_save({ 'data' : {'R_height' : figs}},
                  configuration['reports']['directory_path'])

### Set $\mathcal{R}_{earthquake}$

The original data were taken from http://socr.ucla.edu/docs/resources/SOCR_Data/SOCR_Data_Earthquakes_Over3.html .

In [ ]:
# Load the data
df = pd.read_csv('data/earthquakes.csv', header=0, parse_dates=[0], infer_datetime_format=True)

In [ ]:
# Show the data
df

In [ ]:
# Plot the density of the magnitudes
fig = plt.figure(figsize=(10,7))
df['Mag'].plot.density(linewidth=3, color='navy')
plt.xlabel('Magnitude')
traverse_and_save({ 'data' : {'R_earthquake' : {'magnitude_density_fig' : fig}}},
                  configuration['reports']['directory_path'])

In [ ]:
# Sample from the density (which is clearly not normal)
# to create a number of non-normal samples

samples = []

n_range = range(5, 101, 5)

for n in n_range:
    for i in range(1000):
        sample = list(df.sample(n=n)['Mag'].values)
        samples.append(sample)
        
# Label the samples as non-normal
samples = util.label_samples(samples, 0)

# Save them
describe_and_save(samples, 'R_earthquake', n_range)

# Plot the samples' skewness and kurtosis
desc = inspect_dataset(samples)
plot_skewness_and_kurtosis(desc)

## Further Inspect $G_4$ and $\mathcal{F}$

In [ ]:
G4 = util.separate_by_size(set_C['C-G4'])

F = {code: util.separate_by_size(contaminated_samples[code]) for code in contaminated_samples}

E = util.separate_by_size(set_E)

D = util.separate_by_label_and_size(set_A)

### $G_4$ vs $\mathcal{E}$

In [ ]:
figures['G4-E'] = {}

In [ ]:
for n in G4.keys():
    g4_subgroup = G4[n]
    g4_desc = inspect_dataset(g4_subgroup, statistics=['rs', 'rk'])
    
    e_subgroup = E[n]
    e_desc = inspect_dataset(e_subgroup, statistics=['rs', 'rk'])
    
    fig = plt.figure(figsize=(10, 4))
    
    plt.scatter(g4_desc[0]['rs'],
                g4_desc[0]['rk'],
                alpha=0.2,
                label='Non-normal',
                color='crimson')
    
    plt.scatter(e_desc[1]['rs'],
                e_desc[1]['rk'],
                alpha=0.2,
                label='Normal',
                color='navy')
    plt.legend()
    
    figures['G4-E'][n-1] = fig

### $G_4$ vs Non-normal Subset of $\mathcal{D}$ vs $\mathcal{E}$

In [ ]:
statistics = ['mad', 'sd']

figures['G4-D0'] = {}
for n in G4.keys():
    g4_subgroup = G4[n]
    g4_desc = inspect_dataset(g4_subgroup, statistics=statistics)
    
    d_subgroup = D[0][n - 1]
    d_desc = inspect_dataset(d_subgroup, 
                             labels=[0 for _ in range(len(d_subgroup))],
                             statistics=statistics)
    
    e_subgroup = E[n]
    e_desc = inspect_dataset(e_subgroup, statistics=statistics)
    
    fig = plt.figure(figsize=(10, 4))
    
    plt.scatter(g4_desc[0]['skewness'],
                g4_desc[0]['kurtosis'],
                alpha=0.2,
                label='$G_4$',
                color='crimson')
    
    plt.scatter(d_desc[0]['skewness'],
                d_desc[0]['kurtosis'],
                alpha=0.2,
                label='Training non-normal',
                color='purple')
    
    plt.scatter(e_desc[1]['skewness'],
                e_desc[1]['kurtosis'],
                alpha=0.2,
                label='Normal',
                color='navy')
    
    #plt.ylim(0, 7)
    #plt.xlim(-3, 3)
    plt.legend()
    
    figures['G4-D0'][n-1] = fig

### $\mathcal{F}$ vs $\mathcal{E}$

In [ ]:
figures['F-E'] = {code : {} for code in F}

In [ ]:
for code in F:
    for n in F[code].keys():
        f_subgroup = F[code][n]
        f_desc = inspect_dataset(f_subgroup)
    
        e_subgroup = E[n]
        e_desc = inspect_dataset(e_subgroup)
    
        fig = plt.figure(figsize=(10, 4))
    
        plt.scatter(f_desc[0]['skewness'],
                    f_desc[0]['kurtosis'],
                    alpha=0.2,
                    label=code + ' contamination',
                    color='crimson')
    
        plt.scatter(e_desc[1]['skewness'],
                    e_desc[1]['kurtosis'],
                    alpha=0.2,
                    label='Normal',
                    color='navy')
        plt.legend()
        plt.title(f'$n={n-1}$')
    
        figures['F-E'][code][n-1] = fig
    
    #fig.suptitle(f'$n={n - 1}$')

## Save the figures and changes to the configuration dictionary

In [8]:
# Save the updates to the configuration dictionary
with open(config_path, 'wb') as f:
    pickle.dump(configuration, f)

In [ ]:
# Save the figures
def save_figures(path, dictionary):
    if type(dictionary) is not dict:
        figure = dictionary
        
        path +=  '.pdf'
        
        if 'savefig' in dir(figure):
            figure.savefig(path, bbox_inches='tight')
        else:
            figure.figure.savefig(path, bbox_inches='tight')
    else:
        for key in dictionary:
            save_figures(path + '_' + str(key), dictionary[key])

save_figures(configuration['reports']['directory_path'], {'data' : figures})